In [2]:
import pandas as pd

In [3]:
filepath = '../data/en.openfoodfacts.org.products.pkl'

In [6]:
df = pd.read_pickle('../data/en.openfoodfacts.org.products.pkl')

In [9]:
df.info(verbose = True, null_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1615984 entries, 0 to 1615983
Data columns (total 184 columns):
 #   Column                                      Non-Null Count    Dtype  
---  ------                                      --------------    -----  
 0   code                                        1615984 non-null  object 
 1   url                                         1615984 non-null  object 
 2   creator                                     1615980 non-null  object 
 3   created_t                                   1615984 non-null  int64  
 4   created_datetime                            1615984 non-null  object 
 5   last_modified_t                             1615984 non-null  int64  
 6   last_modified_datetime                      1615984 non-null  object 
 7   product_name                                1526408 non-null  object 
 8   abbreviated_product_name                    922 non-null      object 
 9   generic_name                                109723 non-n

In [12]:
df.countries.value_counts()

France                                             425230
United States                                      309575
en:fr                                              150807
en:es                                               90421
en:france                                           67743
                                                    ...  
France,Guadeloupe,La Réunion,Suisse, en:morocco         1
Canada,France,La Réunion,Suisse                         1
France, Allemagne, Hong Kong, Espagne, en:be            1
Austria,France,Poland,United States                     1
en:France, España                                       1
Name: countries, Length: 9970, dtype: int64